In [0]:
import pandas as pd
import numpy as np
import string
import re
from unicodedata import normalize
from matplotlib import pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
df = pd.read_csv("/content/drive/My Drive/Deep Learning/deueng/deu.txt",encoding='utf-8', delimiter = r'\t', header=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
df.columns = ['English', 'German']

In [0]:
df['German'] = df['German'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [0]:
df['English']  = df['English'].str.replace("won't","will not")
df['English']  = df['English'].str.replace("ain't","am not")
df['English']  = df['English'].str.replace("'s"," is")
df['English']  = df['English'].str.replace("'m"," am")
df['English']  = df['English'].str.replace("'re'"," are")
df['English']  = df['English'].str.replace("can't","can not")
df['English']  = df['English'].str.replace("'ll"," will")
df['English']  = df['English'].str.replace("n't"," not")

In [0]:
df['German'] = df['German'].str.replace("'s"," es")

In [0]:
df['English'] = df['English'].map(lambda x: x.lower())
df['German'] = df['German'].map(lambda x: x.lower())

In [0]:
df['English'] = df['English'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x)) #removing all charecters except alphabets
df['English'] = df['English'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
df['English'] = df['English'].map(lambda x: ' '.join(word for word in x.split() if len(word)>1)) #removing single charecters'

In [0]:
df['German'] = df['German'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x)) #removing all charecters except alphabets
df['German'] = df['German'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
df['German'] = df['German'].map(lambda x: ' '.join(word for word in x.split() if len(word)>1)) #removing single charecters'

In [0]:
df['English'][22519]

'call 110 right now'

In [0]:
df.shape

(192881, 2)

In [0]:
df = df[0:10000]

In [0]:
df_train = df[:9000]
df_test = df[9000:]

In [0]:
english_tokenizer = Tokenizer()
english_tokenizer.fit_on_texts(df['English'])
eng_vocab_size = len(english_tokenizer.word_index)+1
print (eng_vocab_size)
print (max([len(x) for x in df['English'].str.split()]))

2205
5


In [0]:
eng_max_len = (max([len(x) for x in df['English'].str.split()]))
eng_max_len = 10

In [0]:
german_tokenizer = Tokenizer()
german_tokenizer.fit_on_texts(df['German'])
german_vocab_size = len(german_tokenizer.word_index)+1
print (german_vocab_size)
print (max([len(x) for x in df['German'].str.split()]))

3565
10


In [0]:
german_max_len = (max([len(x) for x in df['German'].str.split()]))

In [0]:
trainX = german_tokenizer.texts_to_sequences(df_train['German'])
trainX = pad_sequences(trainX, maxlen=german_max_len, padding='post')

In [0]:
trainX.shape

(9000, 10)

In [0]:
trainY = english_tokenizer.texts_to_sequences(df_train['English'])
trainY = pad_sequences(trainY, maxlen=eng_max_len, padding='post')

In [0]:
trainY.shape

(9000, 10)

In [0]:
trainY = to_categorical(trainY, num_classes=eng_vocab_size)

In [0]:
trainY.shape

(9000, 10, 2205)

In [0]:
testX = german_tokenizer.texts_to_sequences(df_test['German'])
testX = pad_sequences(testX, maxlen=german_max_len, padding='post')
testY = english_tokenizer.texts_to_sequences(df_test['English'])
testY = pad_sequences(testY, maxlen=eng_max_len, padding='post')
testY = to_categorical(testY, num_classes=eng_vocab_size)
testY.shape

(1000, 10, 2205)

In [0]:
embedding_size = 256

In [0]:
def time_distributed_dense(x, w, b=None, dropout=None,
                           input_dim=None, output_dim=None, timesteps=None):
    '''Apply y.w + b for every temporal slice y of x.
    '''
    if not input_dim:
        # won't work with TensorFlow
        input_dim = K.shape(x)[2]
    if not timesteps:
        # won't work with TensorFlow
        timesteps = K.shape(x)[1]
    if not output_dim:
        # won't work with TensorFlow
        output_dim = K.shape(w)[1]

    if dropout:
        # apply the same dropout pattern at every timestep
        ones = K.ones_like(K.reshape(x[:, 0, :], (-1, input_dim)))
        dropout_matrix = K.dropout(ones, dropout)
        expanded_dropout_matrix = K.repeat(dropout_matrix, timesteps)
        x *= expanded_dropout_matrix

    # collapse time dimension and batch dimension together
    x = K.reshape(x, (-1, input_dim))

    x = K.dot(x, w)
    if b:
        x = x + b
    # reshape to 3D tensor
    x = K.reshape(x, (-1, timesteps, output_dim))
    return x

In [0]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent
from keras.layers import TimeDistributed, Dense
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space

        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
model = Sequential()
model.add(Embedding(german_vocab_size,embedding_size,input_length=german_max_len))
model.add(LSTM(100,return_sequences=True))
model.add(AttentionDecoder(100, eng_vocab_size))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 256)           912640    
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 100)           142800    
_________________________________________________________________
AttentionDecoder (AttentionD (None, 10, 2205)          6057230   
Total params: 7,112,670
Trainable params: 7,112,670
Non-trainable params: 0
_________________________________________________________________


In [0]:
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [0]:
model.fit(trainX, trainY, epochs=30, batch_size=128, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Instructions for updating:
Use tf.cast instead.
Train on 9000 samples, validate on 1000 samples
Epoch 1/30
 - 8s - loss: 3.2227 - val_loss: 2.1422

Epoch 00001: val_loss improved from inf to 2.14217, saving model to model.h5
Epoch 2/30
 - 4s - loss: 1.6074 - val_loss: 2.0690

Epoch 00002: val_loss improved from 2.14217 to 2.06901, saving model to model.h5
Epoch 3/30
 - 4s - loss: 1.5311 - val_loss: 2.0278

Epoch 00003: val_loss improved from 2.06901 to 2.02778, saving model to model.h5
Epoch 4/30
 - 4s - loss: 1.4613 - val_loss: 1.9931

Epoch 00004: val_loss improved from 2.02778 to 1.99307, saving model to model.h5
Epoch 5/30
 - 4s - loss: 1.3988 - val_loss: 1.9761

Epoch 00005: val_loss improved from 1.99307 to 1.97611, saving model to model.h5
Epoch 6/30
 - 4s - loss: 1.3522 - val_loss: 1.9300

Epoch 00006: val_loss improved from 1.97611 to 1.92999, saving model to model.h5
Epoch 7/30
 - 4s - loss: 1.3067 - val_loss: 1.9021

Epoch 00007: val_loss improved from 1.92999 to 1.90211, sa

In [0]:
englishword_for_id = dict((v,k) for k,v in english_tokenizer.word_index.items())  
englishword_for_id[1]

'tom'

In [0]:
for word in np.argmax(model.predict(pad_sequences(german_tokenizer.texts_to_sequences(["ich habe gestern einen Film gesehen"]),padding="post",maxlen=10)),axis=2)[0]:
  if word != 0:
    print (englishword_for_id[word])

saw
that


In [0]:
pad_sequences(german_tokenizer.texts_to_sequences(["Guten Morgen Jagan"]),padding="post",maxlen=10)

array([[1165,  424,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [0]:
predict_sentence_split = []
for word in np.argmax(model.predict(pad_sequences(german_tokenizer.texts_to_sequences([df['German'][9999]]),padding="post",maxlen=10)),axis=2)[0]:
  if word != 0:
    print (englishword_for_id[word])
    predict_sentence_split.append(englishword_for_id[word])

found
it


In [0]:
from nltk.translate.bleu_score import sentence_bleu

In [0]:
df['English'][9999]

'just found it'

In [0]:
sentence_bleu([df['English'][9999].split()],predict_sentence_split)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.6065306597126334